## THIS FILE CONTAINS ALL THE FUNCTIONS USED TO IMPORT ROWS AND TRUNCATE TABLES IN THE GENERATES RDBS

In [9]:
import pymysql
endpoint = 'ta10projectinstance1.c0m59tujlbqm.ap-southeast-2.rds.amazonaws.com'
username = 'admin'
password = 'fit5120ta10'
database_name = 'fit5120_i3_schema'
client_flag = 'CLIENT.MULTI_STATEMENTS'

connection = pymysql.connect(host = endpoint, user = username, passwd = password, db = database_name)

# Function used to clear all entries in a single table considering no indexes or Foreign keys are present
def truncate_handler(table):
    cursor = connection.cursor()
    cursor.execute("""TRUNCATE {}""".format(table))
    connection.commit()
    connection.close()
    return 'table {} has been truncated'.format(table)

def country_region_handler(values):
    query = """INSERT INTO country_region (country_region_id, country_code, region_name) VALUES (%s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    connection.close()
    return 'import successful {}'.format(values)

def country_handler(values):
    query = """INSERT INTO country (country_name, country_code) VALUES (%s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    connection.close()
    return 'import successful {}'.format(values)

def symptom_handler(values):
    query = """INSERT INTO symptom (symp_id, symp_name) VALUES (%s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    connection.close()
    return 'import successful {}'.format(values)

def case_details_handler(values):
    query = """INSERT INTO case_details (case_id, symp_id) VALUES (%s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    connection.close()
    return 'import successful {}'.format(values)

def case_handler(values):
    query = """INSERT INTO `case` (case_id, case_status, case_date, case_gender, country_region_id, case_death_date) VALUES (%s, %s, %s, %s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    connection.close()
    return 'import successful {}'.format(values)

def prediction_handler(values):
    query = """INSERT INTO predictions (country_region_id, pred_date, pred_count) VALUES (%s, %s, %s)"""
    cursor = connection.cursor()
    cursor.execute(query, values)
    connection.commit()
    connection.close()
    return 'predictions table has been updated'

def query(query):
    sql_query = """{}""".format(query)
    cursor = connection.cursor()
    cursor.execute(sql_query)
    output = cursor.fetchall()
    print(output)
    connection.close()
    return

def delete_row(table, column, value):
    sql_query = """DELETE FROM `{}` WHERE {} LIKE %{}%""".format(table, column, value)
    cursor = connection.cursor()
    cursor.exectue(sql_query)
    connection.close()
    return 'All rows with {} in column {} has been deleted from table {}'.format(value, column, table)

In [ ]:
# Uploads table for country
c_df = pd.read_csv('sql_country.csv')
for row in range(0,c_df.shape[0]):
    in_tuple = tuple(c_df.iloc[row, :])
    country_handler(in_tuple)

In [ ]:
# Uploads table for country_region
cr_df = pd.read_csv('sql_country_region.csv')
for row in range(0,cr_df.shape[0]):
    in_tuple = tuple(cr_df.iloc[row, :])
    if type(in_tuple[2]) != str:
        country_region_handler((in_tuple[0], in_tuple[1], None))
    else:
        country_region_handler(in_tuple)

In [ ]:
# Uploads table for symptom
s_df = pd.read_csv('sql_symptom.csv')
for row in range(0,s_df.shape[0]):
    in_tuple = tuple(s_df.iloc[row, :])
    symptom_handler(in_tuple)

In [ ]:
# Uploads table for case_details
cd_df = pd.read_csv('sql_case_details.csv')
for row in range(0,cd_df.shape[0]):
    in_tuple = tuple(cd_df.iloc[row, :])
    case_details_handler(in_tuple)

In [ ]:
# Uploads table for case_details
case_df = pd.read_csv('sql_case.csv')
for row in range(0,case_df.shape[0]):
    con1 = False
    con2 = False
    in_tuple = tuple(case_df.iloc[row, :])
    if type(in_tuple[3]) != str:
        con1 = True
    if type(in_tuple[5]) != str:
        con2 = True
    if con1 == True and con2 == True:
        case_handler((in_tuple[0], in_tuple[1], in_tuple[2], None, in_tuple[4], None))
    elif con1 == True and con2 == False:
        case_handler((in_tuple[0], in_tuple[1], in_tuple[2], None, in_tuple[4], in_tuple[5]))
    elif con1 == False and con2 == True:
        case_handler((in_tuple[0], in_tuple[1], in_tuple[2], in_tuple[3], in_tuple[4], None))
    else: 
        case_handler(in_tuple)

In [ ]:
# Closes connection to the rdbs
connection.close()

In [23]:
query("SELECT DISTINCT case_date FROM `case` WHERE STR_TO_DATE(case_date, '%Y-%m-%d') >= STR_TO_DATE('2022-09-22', '%Y-%m-%d')")

(('2022-09-22',),)